# Retrieval Augmentation Generation (RAG) with LLAMA.CPP Quantized Model

### Install llama.cpp llama-cpp-python, chromadb
In my previous video, I have shown how to build a quantized model from llama.cpp

In this notebook, you will see how to do RAG on a quantied model so that you can query your documents.

CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.64 --no-cache-dir

pip install chromadb 

##### Step 1: Instantiate an embed model which later will be used for storing data in the vector DB

In [3]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


embed_model= ".\\models\\embedding\\all-MiniLM-L6-v2"

# embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
embed_model_id = ".\\models\\embedding\\all-MiniLM-L6-v2"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

# embed_model = HuggingFaceEmbeddings(
#     model_name=embed_model_id,
#     model_kwargs={'device': device},
#     encode_kwargs={'device': device, 'batch_size': 32}
# )

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

cuda:0


##### Step 2: Process Custom Content into Chunks

In [4]:

from langchain.document_loaders import WebBaseLoader

##loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
# loader = WebBaseLoader("https://www.quadratics.com/MLOPSimplified.html")
# data = loader.load()

from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader(".//doc", glob="**/*.pdf")
docs = loader.load()
len(docs)



2

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=50)
all_splits = text_splitter.split_documents(docs)

##### Step 3: Store the custom content into a Vector DB (Chroma)

In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embed_model)



C:\Projects\llamacpp_RAG\venv\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


##### Step 4: Set bindings for LLAMA.CPP quantized model and instantiate the model

In [1]:
from langchain.embeddings import LlamaCppEmbeddings
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
n_gpu_layers = 32  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [2]:
#llama = LlamaCppEmbeddings(model_path="/data/llama.cpp/models/llama-2-7b-chat/ggml-model-q4_0.bin")
#mistral working good
# model_path = "./models/llm/mistral-7b-instruct-v0.1.Q6_K.gguf" 
# model_path = "./models/llm/Meta-Llama-3-8B-Instruct.Q4_0.gguf"
# model_path = "./models/llm/falcon2-11B.Q6_K.gguf"

# model_path = "./models/llm/amethyst-13b-mistral.Q8_0.gguf"

model_path = "./models/llm/mistral-7b-instruct-v0.1.Q8_0.gguf"

llm = LlamaCpp(
    model_path = model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=4000,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=False,
)


In [62]:
dialog = f"""
## Speaker diarization and captioning:
**00:00**  Speaker A: This recording is from the British Council.
**00:11**  Speaker A: Hello, Junko Mori speaking. How can I help you? 
**00:16**  Speaker B: Hi, Junko. It's Andrea here from Red Band. I'm calling about our latest order.
**00:22**  Speaker A: Everything arrived okay, right? We got the delivery confirmation at our end. 
**00:28**  Speaker B: Yes, everything's fine with the order. I'm calling about the invoice and the payment terms. I need a favor.
**00:35**  Speaker A: A favor? What do you need? 
**00:38**  Speaker B: This is a little, uh, difficult, but I need an extension on the payment terms. I know they're usually 30 days, but we're having some cash flow problems. You'd really be helping us out if you could extend it to 60 days. 
**00:55**  Speaker A: I'm not sure if I can do that, Andrea. We've got regulations at our end and also have to manage our own cash flow. 
**01:04**  Speaker B: I promise this won't become the norm, Junko. Actually, I also want to place another new order, the same size order as last time. It's for an important customer, and they pay on delivery. 
**01:17**  Speaker A: I see. So your cash flow problem will be solved after this new order is delivered?
**01:23**  Speaker B: Exactly.
**01:25**  Speaker A: That sounds good. Hold on, Andrea. Let me see what I can do. <noise> Yes, I think we can make an exception this time.
**01:36**  Speaker B: That's great, Junko. I appreciate your help.
**01:39**  Speaker A: And we appreciate your business, Andrea. <noise> It works both ways.
**01:45**  Speaker B: Thanks again, Junko. Can you send me a quick email confirmation of the payment terms extension? 
**01:51**  Speaker A: Sure, no problem. We're happy to help you. 
**01:56**  Speaker B: Great, and I'll email you the new order.
**01:59**  Speaker A: Thanks. I'll keep an eye out for it. Talk to you soon.
**02:03**  Speaker B: You too. Goodbye. <noise>
**02:12**  Speaker C: This recording is from the British Council. To find more activities to practise your English, visit www.britishcouncil.org/learnenglish 
"""

In [5]:
prompt = f"""
  You are an auditor, your task is to audit the telemarketer conversation with the customer.
  Given the following conversation between a telemarketer and customer:    
  conversation: 
  {dialog}

  Perform an audit on the above based on the following criteria:
    1. Did the telemarketer address the client/customer name.
    2.Did the telemarketer greet the customer.
    3.Did the telemarketer introduce herself/himself
    4.Did the telemarketer query to get information about the customer.
    5.Did the telemarketer query about the company financial status.
  Give the reasons and result of the above audit a pass or fail.

"""

In [3]:
dialog = f"""
SPEAKER 2 0:00:01 Hello. Hi, is this Sunny Nathans? Yes, speaking. 
SPEAKER 1 0:00:05 Hi, Sunny. This is Mihir calling from IPP. Is this a good time to speak with you? Calling from? IPP. Are you a bit busy, or is this a good time? You're calling from where? I didn't get from where you're calling? Right. It's IPP. So basically, your details are passed on to me by a senior consultant. 
SPEAKER 2 0:00:25 Her name's Elena Prio. 
SPEAKER 1 0:00:26 So she had reached out to you a couple of years ago, and your details were here for me to follow up. So I just wanted to check. Do you recall Elena Prio reaching out to you on LinkedIn, or is that a slip through the cracks? Maybe LinkedIn get a lot of junk in there anyway. I don't remember, but may I know the context of this call? 
SPEAKER 2 0:00:51 Right, Sunny. So very quickly, IPP is in. 
SPEAKER 1 0:00:56 We are Singapore's largest and oldest financial institution, and we specifically help expats and PRs in Singapore with financial advisory. So we do offshore investment planning from unit trusts to insurance products. 
SPEAKER 2 0:01:09 We also do international mortgages, tax-efficient planning, basically an A to Z of financial planning. 
SPEAKER 1 0:01:16 And we sell products from mostly all insurance companies here in Singapore. So that really helps our clients as well. I wanted to check with you. Who might be key? Who might be key to sort of explore our solutions? Maybe a 15 to 20-minute Zoom session with our senior consultant. Would that be of your interest? Not of direct interest, but I'm just, like when you say, like what do you do? Can you just in layman's word, like just explain to me how do you help? 
SPEAKER 2 0:01:49 Right. 
SPEAKER 1 0:01:50 So we have various types of solutions. So we do the most basic things. So health insurance, life insurance. Which I'm not interested at all. 
SPEAKER 2 0:01:59 Right. 
SPEAKER 1 0:02:00 We also do investment planning. So we also do unit trusts. So we have a tie-up with a lot of intermediaries. So fund houses like J.P. Morgan, fund houses like PIMCO, they invest monies in equities and bonds. So we can help in wealth accumulation and wealth creation. Like I said, my senior consultant, Elena Prior, should be in a better position to run you through our exact products. But in any case, if you are in the market, it's a good time because the market is having an optimistic sentiment. But if you're not looking to buy anything, we can still have a chat. We can share with you exactly what we do and exactly how our clients have benefited from this. I don't want to go to a separate call on this. If you just tell me, like what exactly do you do? 
SPEAKER 2 0:02:48 Right. 
SPEAKER 1 0:02:48 So we are a financial institution. We're a financial consulting firm. So we help high-network individuals. We help normal retail customers. We help customers to grow their wealth and to increase returns. Which I get. But when you say like so, is it like you ask for funds or you manage my funds or like what? Like exactly, I'm just trying to figure out what do you have? 
SPEAKER 2 0:03:12 Yes, the management of your funds. 
SPEAKER 1 0:03:14 So you can invest with us and we can cause the returns. 
SPEAKER 2 0:03:17 Invest with you? 
SPEAKER 1 0:03:19 Yes, you can invest your money through us. And then we can sort of help in capital appreciation. Again, I'm getting your, so I have to give you money and you will invest it on my behalf. Or would you like give me guidance on my current portfolios? It can be both. We do both things. So we can also give you a second opinion on existing investments. But you can also invest with us and you can put money through us and grow your capital. But of course, where we put the money will be up to you. Obviously, you will have access to the funds that we invest in. And you can sort of also understand. 
SPEAKER 2 0:03:55 You know, the exact part of the fund. 
SPEAKER 1 0:04:01 I'm not too sure. Just drop me an email and if I'm interested, I'll get back to you. 
SPEAKER 2 0:04:07 Right, right. 
SPEAKER 1 0:04:08 How would you, would you be interested to jump on a Zoom call with my consultant? Because she's a subject matter. Yeah, I don't have time to do that. And I think I've seen like multiple emails from her on my LinkedIn and I didn't realize. 
SPEAKER 2 0:04:22 But I'm not clear. 
SPEAKER 1 0:04:23 Like I get so many of such similar emails. But if you can just tell me exactly what you guys do and like if I find it interesting. But right now, it just seems like a spam kind of thing. 
SPEAKER 2 0:04:34 I understand, I understand. 
SPEAKER 1 0:04:36 So the reason I'm asking for a Zoom call is because she's a subject matter expert on financials. 
SPEAKER 2 0:04:43 I'm more of a client acquisition. I don't know. 
SPEAKER 1 0:04:51 On a Saturday noon, like 20 minutes max of your time. 
SPEAKER 2 0:04:55 I'm there. I'm in a position to help you. 
SPEAKER 1 0:04:58 Yeah, I've been traveling and I'm traveling from this Saturday again for the whole week. 
SPEAKER 2 0:05:03 Exactly next week. 
SPEAKER 1 0:05:05 And then again, so if you can just drop me an email. But like step, I'll go through the email. I'll get back to you so that we don't waste your time also. Like key thing that I want to know. I just want to know how do you add value. 
SPEAKER 2 0:05:20 Okay. And what are the mechanics of it? Right. 
SPEAKER 1 0:05:23 I don't want to. I don't want. I don't want to do another insurance or a health insurance advice. 
SPEAKER 2 0:05:29 Okay. 
SPEAKER 1 0:05:29 Like I don't want. This is not health insurance. If you don't want health insurance, we can also do other. I don't want life insurance. I just want to know exactly like what, how do you help? 
SPEAKER 2 0:05:39 Like one, two, three. 
SPEAKER 1 0:05:41 And in fact, to be honest, I am looking for a financial consultant to manage both for me and in India. So it's not like I'm not. But I do want to know. But in that process, I'm going through so many. 
SPEAKER 2 0:05:55 I'm going through so many junk discussions. That's like just bothering. I understand. Okay. 
SPEAKER 1 0:06:00 How, how would I do this? How about I ask Elena prior to give you a voice call? She'll be in a very much better place to actually address your exact concern. No, I, I, I told you, like you've gotten on the call right now. Just sent me an email. 
SPEAKER 2 0:06:13 If it is of interest. I'll get back to you. 
SPEAKER 1 0:06:15 And I don't want like a massive. So just like five lines would do on what do you guys do? 
SPEAKER 2 0:06:20 Just that. Okay. First of all. 
SPEAKER 1 0:06:23 What do you think? 
SPEAKER 2 0:06:24 Could I get your email? 
SPEAKER 1 0:06:25 Sort of send some information on that? 
SPEAKER 2 0:06:28 No. 
SPEAKER 1 0:06:29 But please don't send me a brochure with like hundreds. 
SPEAKER 2 0:06:31 It's not like I'm not a corporate company. I think just like five sentences, four sentences, like which tells what you guys do. So that next time, like, and if it's of interest, I'll get back to Elena or you like, as a try to the corner and he could have, we could set up a zoom call and we take it forward. Got it. 
SPEAKER 1 0:06:50 Got it. 
SPEAKER 2 0:06:51 No worries. Okay. 
SPEAKER 1 0:06:53 Could I get your email? 
SPEAKER 2 0:06:54 Sure. I'll send it there. Sure. It's an online. J. M. A. T. H. E. W. S. Okay. So it's funny. J. Matthews. Yeah. All right. I'll send you an email by today so you can review it at your time. Okay. Thanks. Thank you so much for your call. Bye. Bye. Bye. Bye. Bye. Bye. Bye. 
 """

In [6]:
prompt = f"""
  You are an auditor, your task is to audit the telemarketer conversation with the customer.
  Given the following conversation between a telemarketer and customer:    
  conversation: 
  {dialog}

  Perform an audit on the above based on the following criteria:
    1. Did the telemarketer address the client/customer name.
    2.Did the telemarketer greet the customer.
    3.Did the telemarketer introduce herself/himself
    4.Did the telemarketer query to get information about the customer.
    5.Did the telemarketer share about the company product to the customer.
    6.Did the telemarketer try to get an appoint to meet the customer.
  Give the reasons and result of the above audit a pass or fail.

"""

In [79]:
prompt = f"""
    you are a good text summariser. Understand the given context and perform the summarisation in \
    meaning manner.
    Below is the text contain information of dialog between the telemarketer and the customer.
    {dialog}

   Summarise on the above dialog between 2 person in the paragraph format.
"""

In [7]:
llm.invoke(prompt)

  The audit is as follows:
  
  Criteria | Result | Reasoning
           --- | --- | ---
          1| Fail| The telemarketer did not address the customer by their name until 2 minutes and 49 seconds into the call, which was after a significant portion of the conversation had already taken place.
          2| Pass| The telemarketer greeted the customer at the beginning of the call.
          3| Pass| The telemarketer introduced herself/herself at the beginning of the call.
          4| Fail| The telemarketer did not query to get information about the customer until 2 minutes and 58 seconds into the call, which was after a significant portion of the conversation had already taken place.
          5| Pass| The telemarketer shared about the company product to the customer throughout the call.
          6| Fail| The telemarketer did not try to get an appointment to meet the customer until 10 minutes and 55 seconds into the call, which was very late in the conversation and may have missed an

'  The audit is as follows:\n  \n  Criteria | Result | Reasoning\n           --- | --- | ---\n          1| Fail| The telemarketer did not address the customer by their name until 2 minutes and 49 seconds into the call, which was after a significant portion of the conversation had already taken place.\n          2| Pass| The telemarketer greeted the customer at the beginning of the call.\n          3| Pass| The telemarketer introduced herself/herself at the beginning of the call.\n          4| Fail| The telemarketer did not query to get information about the customer until 2 minutes and 58 seconds into the call, which was after a significant portion of the conversation had already taken place.\n          5| Pass| The telemarketer shared about the company product to the customer throughout the call.\n          6| Fail| The telemarketer did not try to get an appointment to meet the customer until 10 minutes and 55 seconds into the call, which was very late in the conversation and may have

##### Step 5: Do a similarity search on the Vectordb to retrieve data related to the query

In [ ]:
question = "what accelerators did quadratic build"
docs = vectorstore.similarity_search(question)
#result = llm_chain(docs)
docs

In [32]:
question = "what is learning buddy?"
docs = vectorstore.similarity_search(question)
#result = llm_chain(docs)
docs

[Document(page_content='How does Learning buddy answer the query?** LLM', metadata={'source': 'doc\\LearningBuddyFAQ.pdf'}),
 Document(page_content='does Learning Buddy work?** Students can interact', metadata={'source': 'doc\\LearningBuddyFAQ.pdf'}),
 Document(page_content='learning buddy?** Yes, users have the ability to', metadata={'source': 'doc\\LearningBuddyFAQ.pdf'}),
 Document(page_content='**2. How does Learning Buddy work?** Students can', metadata={'source': 'doc\\LearningBuddyFAQ.pdf'})]

##### Step 6: Create a RAG pipeline to contextualize with the custom data and Query

In [33]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

In [37]:
rag_pipeline("What is SQL Select for? give a example")

 SQL SELECT statement is used to retrieve data from one or multiple tables in a database based on specified conditions. Here is an example of how it can be used:
```sql
SELECT column1, column2, column3
FROM table_name
WHERE condition;
```
This query retrieves the values of column1, column2 and column3 from table\_name where the condition is true.

Helpful Answer: SQL SELECT statement is used to retrieve data from one or multiple tables in a database based on specified conditions. Here is an example of how it can be used:
```sql
SELECT column1, column2, column3
FROM table_name
WHERE condition;
```
This query retrieves the values of column1, column2 and column3 from table\_name where the condition is true.

{'query': 'What is SQL Select for? give a example',
 'result': ' SQL SELECT statement is used to retrieve data from one or multiple tables in a database based on specified conditions. Here is an example of how it can be used:\n```sql\nSELECT column1, column2, column3\nFROM table_name\nWHERE condition;\n```\nThis query retrieves the values of column1, column2 and column3 from table\\_name where the condition is true.\n\nHelpful Answer: SQL SELECT statement is used to retrieve data from one or multiple tables in a database based on specified conditions. Here is an example of how it can be used:\n```sql\nSELECT column1, column2, column3\nFROM table_name\nWHERE condition;\n```\nThis query retrieves the values of column1, column2 and column3 from table\\_name where the condition is true.'}

In [ ]:
rag_pipeline("how do the accelerators built by Quadratic help their customers")

In [ ]:
llm("what accelerators did quadratic build")